In [1]:
import os
import sys
import pandas as pd
sys.path.append("../ixf2json/")
import ixf2json

import pyodbc
import json
import sqlalchemy as sa

In [2]:
from IPython.core.display import display, HTML, Markdown
display(HTML("<style>.container { width:95% !important; }</style>"))

In [3]:
# # Uncomment and run these shell commands to download and extract the sample database files
# ! wget ftp://ftp.software.ibm.com/software/data/sample/GSDB_DB2_LUW_ZOS_v2r3.zip -P ../ 
# ! unzip ../GSDB_DB2_LUW_ZOS_v2r3.zip -d ../

In [4]:
os.listdir("..")

['example_notebooks',
 'json_data',
 'README.md',
 '.gitignore',
 'ixf2json',
 '.ipynb_checkpoints',
 'DB2',
 'environment.yaml',
 '.git',
 'cp4d-mortgage-artifacts',
 'creds']

In [4]:
# os.listdir("../json_data/")

In [22]:
log = {
    "json_conversion_success": [],
    "file_name_errors": [], 
    "file_conversion_errors": [],
    "file_extension_errors": [],
    "json_read_errors": [],
    "sql_insert_errors": []
    }


In [5]:
log = {
    "json_conversion_success": [],
    "file_name_errors": [], 
    "file_conversion_errors": [],
    "file_extension_errors": [],
    "json_read_errors": [],
    "sql_insert_errors": []
    }

def convert_ixf():
    for file in sorted(os.listdir("../DB2/data")):
        file_list = file.split(".")
        if len(file_list) > 2:
            print("file name error: ", file)
            log["file_name_errors"].append(file)
        else:
            filename = file_list[0]
            extension = file_list[-1]
            if extension != "ixf":
                print("file extension error: ", file)
                log["file_extension_errors"].append(file)
            else:
                try:
                    ixf2json.convert(f'../DB2/data/{file}', f'../json_data/{filename}.json')
                    log["json_conversion_success"].append(file)
                except Exception as e:
                    log["file_conversion_errors"].append({"file": file, "error_mssg": str(e)})
                    pass
        
    with open("log.json", "w") as f:
        json.dump(log, f)

In [6]:
convert_ixf()

file extension error:  EXPORT.out
file extension error:  GOSALES_VERSION.txt
file extension error:  db2move.lst
file extension error:  gsdb_constraints.sql
file extension error:  gsdb_readme.txt
file name error:  tab110a.001.xml
file name error:  tab9a.001.lob


In [15]:
with open('log.json', 'r') as f:
     log = json.load(f)

In [16]:
log.keys()

dict_keys(['json_conversion_success', 'file_name_errors', 'file_conversion_errors', 'file_extension_errors', 'json_read_errors', 'sql_insert_errors'])

In [17]:
log["file_name_errors"]

[]

In [18]:
len(log["json_conversion_success"])

0

In [19]:
log["file_conversion_errors"]

[]

In [20]:
json_tables = os.listdir("../json_data/")

In [13]:
ixf_tables = os.listdir("../DB2/data")

In [14]:
len(ixf_tables)

158

In [15]:
len(json_tables)

151

In [16]:
for record in log["file_conversion_errors"]:
    file_lst = record["file"].split(".")
    extension = file_lst[-1]

    if extension == "ixf":
        filename = file_lst[0]
        print(filename)
        with open(f"../json_data/{filename}.json", "r") as f:
            json_obj = json.load(f)
        print(filename, len(json_obj))

## Parsing list that maps filenames to table/schema names

In [5]:
ref_lst = []

schema_set = set()

with open("../DB2/data/db2move.lst", "r") as f:
    for line in f.readlines():
        lst = line.split("!")
        schema, table = lst[1].split(".")
        schema = schema.replace('"', '' ).strip()
        table = table.replace('"', '' ).strip()
        file = lst[2]
        filename = lst[2].split(".")[0]
        result = dict(
            schema=schema,
            table=table,
            file=file,
            filename=filename
        )
        schema_set.add(schema)
        ref_lst.append(result)

In [6]:
ref_df = pd.DataFrame(ref_lst)

In [7]:
ref_df.shape

(151, 4)

In [8]:
ref_df.head()

,schema,table,file,filename
0,GOSALESRT,ACTIVITY_STATUS_LOOKUP,tab1.ixf,tab1
1,GOSALES,BRANCH,tab2.ixf,tab2
2,GOSALES,CONVERSION_RATE,tab3.ixf,tab3
3,GOSALES,COUNTRY,tab4.ixf,tab4
4,GOSALES,CURRENCY_LOOKUP,tab5.ixf,tab5


In [9]:
json_tables = os.listdir("../json_data/")

In [10]:
json_tables.sort(key=lambda x: int(x.strip("tab").strip(".json")))

In [11]:
len(json_tables)

151

In [12]:
schema_set

{'GOSALES', 'GOSALESCT', 'GOSALESDW', 'GOSALESHR', 'GOSALESMR', 'GOSALESRT'}

In [13]:
ref_df

,schema,table,file,filename
0,GOSALESRT,ACTIVITY_STATUS_LOOKUP,tab1.ixf,tab1
1,GOSALES,BRANCH,tab2.ixf,tab2
2,GOSALES,CONVERSION_RATE,tab3.ixf,tab3
3,GOSALES,COUNTRY,tab4.ixf,tab4
4,GOSALES,CURRENCY_LOOKUP,tab5.ixf,tab5
...,...,...,...,...
146,GOSALES,TIME_QUARTER_LOOKUP,tab147.ixf,tab147
147,GOSALESHR,TRAINING,tab148.ixf,tab148
148,GOSALESHR,TRAINING_DETAILS,tab149.ixf,tab149
149,GOSALES,XGOREV,tab150.ixf,tab150


## Read in JSONs into pandas, display and record metadata, log errors

In [23]:
ref_df["col_count"] = pd.NA
ref_df["row_count"] = pd.NA

for json_table in json_tables:
    filename = json_table.split(".")[0]
#     display(Markdown(f"### {filename}"))
    try:
        df = pd.read_json(f"../json_data/{json_table}", lines=False)
        row_ct, col_ct = df.shape
#         display(Markdown(f"#### Size: {df_shape}"))
        ref_df.loc[ref_df["filename"] == filename, ["row_count"]] = row_ct
        ref_df.loc[ref_df["filename"] == filename, ["col_count"]] = col_ct

    except:
        display(Markdown(f"#### Error reading file {json_table}"))
        log["json_read_errors"].append(json_table)
        pass

#### Error reading file tab110.json

In [24]:
ref_df.sort_values("row_count", ascending=False).head(10)

,schema,table,file,filename,col_count,row_count
140,GOSALESDW,SLS_SALES_ORDER_DIM,tab141.ixf,tab141,4,446023
87,GOSALES,ORDER_DETAILS,tab88.ixf,tab88,9,446023
139,GOSALESDW,SLS_SALES_FACT,tab140.ixf,tab140,17,446023
141,GOSALESDW,SLS_SALES_TARG_FACT,tab142.ixf,tab142,8,233625
128,GOSALES,SALES_TARGET,tab129.ixf,tab129,9,233625
79,GOSALESDW,MRK_PRODUCT_SURVEY_FACT,tab80.ixf,tab80,6,165074
101,GOSALESMR,PRODUCT_SURVEY_RESULTS,tab102.ixf,tab102,6,165074
59,GOSALESDW,FIN_FINANCE_FACT,tab60.ixf,tab60,6,164132
97,GOSALES,PRODUCT_FORECAST,tab98.ixf,tab98,7,129096
19,GOSALESDW,DIST_PRODUCT_FORECAST_FACT,tab20.ixf,tab20,7,129096


In [21]:
bad_jsons = set(log["json_read_errors"])

In [22]:
bad_jsons

{'tab110.json'}

## Insert data into SQL DB

### Make sql connection
You will need to change the connection details in `creds.json` in order to connect to your own SQL instance.

In [27]:
! ls ../

DB2                     creds                   ixf2json
README.md               environment.yaml        json_data
cp4d-mortgage-artifacts example_notebooks


In [45]:
with open("../creds/creds.json", "r") as f:
    all_creds = json.load(f)

all_creds.keys()

creds = all_creds["ms-sqlserver-openshift"]

In [35]:
creds = dict(
    username="sa",
    password="P@ssw0rd",
    host="127.0.0.1",
    port=1433,
    db="tempdb"
)

In [46]:
drivers = pyodbc.drivers()
driver = drivers[0]

with open("../creds/creds.json", "r") as f:
    all_creds = json.load(f)

# creds = all_creds["ms-sqlserver"]
locals().update(creds)  # Creating local variables from the JSON entries for this database

connxn_string = f"mssql+pyodbc://{username}:{password}@{host}:{port}/{db}?driver={driver}"

In [47]:
connxn_string

'mssql+pyodbc://sa:P@ssw0rd@mssql-service-mssql.apps.test-dbs.ocp.csplab.local:31433/tempdb?driver=ODBC Driver 17 for SQL Server'

In [48]:
# import sqlalchemy_utils as sa_utils
# sa_utils.functions.drop_database(connxn_string)

In [49]:
engine = sa.create_engine(
    connxn_string, 
    connect_args={'autocommit':True},  # This is necessary for some reason.
    fast_executemany=True  # This provides a substantial speed-up for query execution.
)

In [50]:
insp = sa.inspect(engine)


In [51]:
db_list = insp.get_schema_names()
print(db_list)

['db_accessadmin', 'db_backupoperator', 'db_datareader', 'db_datawriter', 'db_ddladmin', 'db_denydatareader', 'db_denydatawriter', 'db_owner', 'db_securityadmin', 'dbo', 'guest', 'INFORMATION_SCHEMA', 'sys']


In [52]:
with engine.connect() as con:
    con.execute("USE master;") 
    con.execute("CREATE DATABASE GSDB;")
    con.execute("USE GSDB;")

In [53]:
# Create Schemata
with engine.connect() as con:
    for schema in schema_set:  
        con.execute(f"CREATE SCHEMA {schema};")

### Insert data using SQL Alchemy and Pandas

In [54]:
def insert_tables():
    log["sql_insert_errors"] = []
    for json_table in json_tables:
        # read in JSON into pandas, display metadata
        filename = json_table.split(".")[0]
        display(Markdown(f"### {filename}"))
        if json_table not in bad_jsons:
            df = pd.read_json(f"../json_data/{json_table}")
        
        num_cols = df.shape[1]  
        tsql_chunksize = 2097 // num_cols  # limit based on sp_prepexec parameter count
        tsql_chunksize = 1000 if tsql_chunksize > 1000 else tsql_chunksize  # cap at 1000

        # Lookup table name and schema name from reference df
        ref_row = ref_df.query("filename == @filename")
        schema = ref_row["schema"].values[0]
        table = ref_row["table"].values[0]
        print(f"{schema}.{table}")
        print(df.shape)
        
        type_mapping = {col : sa.types.NVARCHAR for col in df.select_dtypes("O")}
        
        # Write df to sql
        try:
            df.to_sql(
                con=engine,
                name=table, 
                schema=schema,
                if_exists="replace",
                method="multi",
                index=False,
                dtype=type_mapping,
                chunksize=tsql_chunksize
                     )
        except Exception as e:
            display(Markdown("# ERROR"))
            log["sql_insert_errors"].append({"json_table": json_table, "error_mssg": str(e)})
            pass

        print("*" * 20)

    with open("log.json", "w") as f:
            json.dump(log, f)

In [55]:
insert_tables()

### tab83

GOSALESDW.MRK_PROMOTION_FACT
(11034, 13)
********************


### tab101

GOSALES.PRODUCT_SIZE_LOOKUP
(55, 24)
********************


### tab140

...

GOSALESDW.SLS_PRODUCT_TYPE_LOOKUP
(21, 24)
********************


### tab43

GOSALESDW.EMP_SUCCESSION_FACT
(181, 7)
********************


### tab14

GOSALESCT.CUST_ORDER_HEADER
(39389, 13)
********************


## Check to make sure it worked

In [56]:
for json_table in json_tables:
    # read in JSON into pandas, display metadata
    filename = json_table.split(".")[0]
    display(Markdown(f"#### {filename}"))
    # if json_table not in bad_jsons:
    # Lookup table name and schema name from reference df
    ref_row = ref_df.query("filename == @filename")
    schema = ref_row["schema"].values[0]
    table = ref_row["table"].values[0]
    print(f"{schema}.{table}")
    if json_table not in bad_jsons:
        df = pd.read_sql_table(table, con=engine, schema=schema)
        print(df.shape)
        display(df.head())
    print("*" * 90)

#### tab83

GOSALESDW.MRK_PROMOTION_FACT


OperationalError: (pyodbc.OperationalError) ('08S01', '[08S01] [Microsoft][ODBC Driver 17 for SQL Server]Communication link failure (0) (SQLPrepare)')
[SQL: SELECT [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] 
FROM [INFORMATION_SCHEMA].[TABLES] 
WHERE [INFORMATION_SCHEMA].[TABLES].[TABLE_SCHEMA] = CAST(? AS NVARCHAR(max)) AND [INFORMATION_SCHEMA].[TABLES].[TABLE_TYPE] = CAST(? AS NVARCHAR(max)) ORDER BY [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME]]
[parameters: ('GOSALESDW', 'BASE TABLE')]
(Background on this error at: http://sqlalche.me/e/13/e3q8)

## Troubleshooting

In [61]:
log.keys()

dict_keys(['json_conversion_success', 'file_name_errors', 'file_conversion_errors', 'file_extension_errors', 'json_read_errors', 'sql_insert_errors'])

In [62]:
for record in log["sql_insert_errors"]:
    print(record["json_table"], record["error_mssg"][:200])
    print("\n")

In [63]:
log["file_conversion_errors"]

[]

In [170]:
filename = "tab129"

In [171]:
json_table = filename + ".json"

In [156]:
df = pd.read_json(f"../json_data/{json_table}")

In [78]:
df.head()

,PRODUCT_LINE_CODE,PRODUCT_LINE_EN,PRODUCT_LINE_FR,PRODUCT_LINE_DE,PRODUCT_LINE_NL,PRODUCT_LINE_SC,PRODUCT_LINE_KO,PRODUCT_LINE_JA,PRODUCT_LINE_CS,PRODUCT_LINE_HU,...,PRODUCT_LINE_FI,PRODUCT_LINE_IT,PRODUCT_LINE_NO,PRODUCT_LINE_PL,PRODUCT_LINE_RU,PRODUCT_LINE_SV,PRODUCT_LINE_ES,PRODUCT_LINE_TH,PRODUCT_LINE_MS,PRODUCT_LINE_ID
0,994,Outdoor Protection,Articles de protection,Outdoor-Schutzausrüstung,Buitensport - preventie,户外防护用品,야외 보호 장비,アウトドア用保護用品,Vybavení do přírody,Védőfelszerelés,...,Ulkoiluvarusteet,Protezione personale,Utendørs beskyttelse,Wyposażenie ochronne,Средства защиты,Skyddsartiklar,Protección aire libre,สิ่งป้องกันเมื่ออยู่กลางแจ้ง,Perlindungan Luar Bangunan,Perlindungan Luar Ruang
1,991,Camping Equipment,Matériel de camping,Campingausrüstung,Kampeerbenodigdheden,露营装备,캠핑 장비,キャンプ用品,Vybavení pro kempování,Kempingfelszerelés,...,Retkeilyvarusteet,Attrezzatura per campeggio,Campingutstyr,Ekwipunek kempingowy,Снаряжение для туризма,Campingutrustning,Equipo de acampada,อุปกรณ์ตั้งแคมป์,Kelengkapan Berkhemah,Perlengkapan Berkemah
2,995,Golf Equipment,Matériel de golf,Golfausrüstung,Golfartikelen,高尔夫球装备,골프 장비,ゴルフ用品,Golfové potřeby,Golffelszerelés,...,Golf-varusteet,Attrezzatura da golf,Golfutstyr,Ekwipunek golfowy,Снаряжение для гольфа,Golfutrustning,Equipo de golf,อุปกรณ์กอล์ฟ,Kelengkapan Golf,Perlengkapan Golf
3,993,Personal Accessories,Accessoires personnels,Accessoires,Persoonlijke accessoires,个人附件,개인 용품,個人装備,Věci osobní potřeby,Személyes kiegészítők,...,Henkilökohtaiset tarvikkeet,Accessori personali,Personlig utrustning,Akcesoria osobiste,Личные принадлежности,Personliga tillbehör,Accesorios personales,อุปกรณ์ส่วนตัว,Aksesori Diri,Aksesori pribadi
4,992,Mountaineering Equipment,Matériel de montagne,Bergsteigerausrüstung,Bergsportartikelen,登山装备,등산 장비,登山用品,Horolezecké vybavení,Hegymászó-felszerelés,...,Vuorikiipeilyvarusteet,Attrezzatura per alpinismo,Klatreutstyr,Sprzęt wspinaczkowy,Горное снаряжение,Klätterutrustning,Equipo de montañismo,อุปกรณ์ปีนเขา,Kelengkapan Mendaki Gunung,Perlengkapan Pendaki Gunung


In [174]:
df.shape

(233625, 9)

In [176]:
ref_row = ref_df.query("filename == @filename")
schema = ref_row["schema"].values[0]
display(Markdown(f"#### Schema: {schema}"))
table = ref_row["table"].values[0]
display(Markdown(f"#### Table Name: {table}"))

#### Schema: GOSALES

#### Table Name: SALES_TARGET